In [30]:
import os
import json
from sqlalchemy import create_engine
import pandas as pd
import great_expectations as ge
from great_expectations.core.expectation_configuration import ExpectationConfiguration

In [2]:
file_path = '../Credentials/keys.json'
if os.path.exists(file_path):
    with open(file_path, 'r') as json_file:
        data = json.load(json_file)
        user = data["user"]
        password = data["password"]
        port = data["port"]
        server = data["server"]
        db = data["db"]
else:
    print(f"File '{file_path}' not found.")
db_connection = f"postgresql://{user}:{password}@{server}:{port}/{db}"
engine = create_engine(db_connection)
print(f"connected!")

connected!


In [3]:
engine = create_engine(db_connection) 
connection = engine.connect() 
table_name = 'jobslinkedin'  
df = pd.read_sql_table(table_name, connection)
Linkedin=df

In [4]:
# Creamos un dataset de GX a partir del DataFrame
linkedin_gx=ge.from_pandas(Linkedin)

In [5]:
Linkedin.head(5)

,job_id,company_id,title,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,...,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,scraped
0,3757940104,553718.0,Hearing Care Provider,Overview\n\nHearingLife is a national hearing ...,NaN,5250.0,NaN,MONTHLY,Full-time,"Little River, SC",...,NaN,Entry level,None,169909000000000000,careers-demant.icims.com,0,FULL_TIME,USD,BASE_SALARY,1699138101
1,3757940025,2192142.0,Shipping & Receiving Associate 2nd shift (Beav...,Metalcraft of Mayville\nMetalcraft of Mayville...,NaN,NaN,NaN,None,Full-time,"Beaver Dam, WI",...,NaN,None,None,169908000000000000,www.click2apply.net,0,FULL_TIME,None,None,1699085420
2,3757938019,474443.0,"Manager, Engineering",\nThe TSUBAKI name is synonymous with excellen...,NaN,NaN,NaN,None,Full-time,"Bessemer, AL",...,NaN,None,Bachelor's Degree in Mechanical Engineering pr...,169908000000000000,www.click2apply.net,0,FULL_TIME,None,None,1699085644
3,3757938018,18213359.0,Cook,descriptionTitle\n\n Looking for a great oppor...,NaN,2227.0,NaN,HOURLY,Full-time,"Aliso Viejo, CA",...,NaN,Entry level,None,169908000000000000,jobs.apploi.com,0,FULL_TIME,USD,BASE_SALARY,1699087461
4,3757937095,437225.0,Principal Cloud Security Architect (Remote),"Job Summary\nAt iHerb, we are on a mission to ...",275834.0,NaN,205956.0,YEARLY,Full-time,United States,...,NaN,Mid-Senior level,None,169909000000000000,careers.iherb.com,0,FULL_TIME,USD,BASE_SALARY,1699085346


In [6]:
Linkedin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33246 entries, 0 to 33245
Data columns (total 28 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   job_id                      33246 non-null  int64  
 1   company_id                  32592 non-null  float64
 2   title                       33246 non-null  object 
 3   description                 33245 non-null  object 
 4   max_salary                  11111 non-null  float64
 5   med_salary                  2241 non-null   float64
 6   min_salary                  11111 non-null  float64
 7   pay_period                  13352 non-null  object 
 8   formatted_work_type         33246 non-null  object 
 9   location                    33246 non-null  object 
 10  applies                     16238 non-null  float64
 11  original_listed_time        33246 non-null  int64  
 12  remote_allowed              4802 non-null   float64
 13  views                       258

In [7]:
Linkedin.columns

Index(['job_id', 'company_id', 'title', 'description', 'max_salary',
       'med_salary', 'min_salary', 'pay_period', 'formatted_work_type',
       'location', 'applies', 'original_listed_time', 'remote_allowed',
       'views', 'job_posting_url', 'application_url', 'application_type',
       'expiry', 'closed_time', 'formatted_experience_level', 'skills_desc',
       'listed_time', 'posting_domain', 'sponsored', 'work_type', 'currency',
       'compensation_type', 'scraped'],
      dtype='object')

In [32]:
# Define the expectations
expectations = []

# Expects that the numeric columns do not have null values.
numerical_columns = ['max_salary', 'med_salary', 'min_salary', 'applies', 'views']
for column in numerical_columns:
    expectation = ExpectationConfiguration(
        expectation_type="expect_column_values_to_not_be_null",
        kwargs={"column": column}
    )
    expectations.append(expectation)

# Expect the salary columns to be greater than 0.
salary_columns = ['max_salary', 'med_salary', 'min_salary']
for column in salary_columns:
    expectation = ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_greater_than",
        kwargs={"column": column, "value": 0}
    )
    expectations.append(expectation)

# Expects text columns to have no null values
text_columns = ['title', 'description', 'location', 'skills_desc']
for column in text_columns:
    expectation = ExpectationConfiguration(
        expectation_type="expect_column_values_to_not_be_null",
        kwargs={"column": column}
    )
    expectations.append(expectation)

# Expects URLs to be valid
url_columns = ['job_posting_url', 'application_url']
url_regex = r'^https?://.+'
for column in url_columns:
    expectation = ExpectationConfiguration(
        expectation_type="expect_column_values_to_match_regex",
        kwargs={"column": column, "regex": url_regex}
    )
    expectations.append(expectation)

# Expects the dates to be valid
date_columns = ['original_listed_time', 'expiry', 'closed_time', 'listed_time']
date_format = "%Y-%m-%d %H:%M:%S"
for column in date_columns:
    expectation = ExpectationConfiguration(
        expectation_type="expect_column_values_to_match_strftime_format",
        kwargs={"column": column, "strftime_format": date_format}
    )
    expectations.append(expectation)

# Create a suite of expectations
suite_name = "linkedin_suite"
suite = ge.core.ExpectationSuite(expectation_suite_name=suite_name)
suite.expectations.extend(expectations)


# Validate the dataset against the suite of expectations
result = linkedin_gx.validate(expectation_suite=suite)

# Save the results in a text file
result_output_path = "validation_results_linkedin.txt"
with open(result_output_path, 'w') as f:
    for idx, res in enumerate(result["results"]):
        expectation_config = res["expectation_config"]
        success = res["success"]
        f.write(f"Expectation {idx + 1}: {expectation_config['expectation_type']}\n")
        f.write(f"Column: {expectation_config['kwargs'].get('column', 'N/A')}\n")
        if "value" in expectation_config["kwargs"]:
            f.write(f"Value: {expectation_config['kwargs']['value']}\n")
        if "regex" in expectation_config["kwargs"]:
            f.write(f"Regex: {expectation_config['kwargs']['regex']}\n")
        if "strftime_format" in expectation_config["kwargs"]:
            f.write(f"Strftime Format: {expectation_config['kwargs']['strftime_format']}\n")
        f.write(f"Success: {success}\n")
        if not success:
            f.write("Result details:\n")
            f.write(json.dumps(res["result"], indent=2))
            f.write("\n")
        f.write("-" * 40)
        f.write("\n")

print("Validation results saved to:", result_output_path)

Validation results saved to: validation_results_linkedin.txt
